In [ ]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen
from tqdm import tqdm_notebook
import unidecode


In [ ]:
df_debut = pd.read_csv("../data/debut_album_1118-utf8.csv")

In [ ]:
def get_genius(data):
    
    df_articles = pd.DataFrame(columns = ['artist', 'album', 'genius_frequency'])
    
    for i in tqdm_notebook(range(len(list(df_debut['artist'])))):

        artist = df_debut['artist'][i]
        album = df_debut['album'][i]
        
        artist_ = artist.replace(" ", "+")
        album_ = album.replace(" ", "+")
        
        page_num = 1
        articles = 0

        while True:

            url_ = 'https://genius.com/api/search/article?page=' + str(page_num) + '&q=' + artist_ + "+" + album_
            response = requests.get(url_).json()

            articles += len(response['response']['sections'][0]['hits'])

            page_num += 1

            if response['response']['next_page'] == None :
                break

        data = {
            'artist': artist,
            'album' : album,
            'genius_frequency' : articles
        }

#         print(data)

        df_articles.loc[len(df_articles)] = data
    
    return df_articles

In [ ]:
genius_articles = get_genius(df_debut)

In [ ]:
print(genius_articles.shape)
genius_articles.tail()

In [ ]:
def decodeuni(str):
    return unidecode.unidecode(str)

In [ ]:
genius_articles['artist'] = genius_articles['artist'].apply(decodeuni)
genius_articles['album'] = genius_articles['album'].apply(decodeuni)
genius_articles['artist'] = genius_articles['artist'].apply(lambda x : x.replace(",", ""))
genius_articles['album'] = genius_articles['album'].apply(lambda x : x.replace(",", ""))

In [ ]:
genius_articles.loc[2, 'album'] = '9am to 5pm 5pm to Whenever'
genius_articles.loc[529, 'album'] = 'If You Love Me'
genius_articles.loc[1144, 'album'] = 'Gover Tela Body Language'
genius_articles.loc[1413, 'album'] = 'Fresh Filter Volumn 1'

In [ ]:
genius_articles.shape

In [ ]:
genius_articles.to_csv("../data/buzz_genius.csv", index=False, encoding='UTF-8')

In [ ]:
df = pd.read_csv("../data/buzz/buzz_genius.csv")

In [ ]:
df.shape

### Upload to MySQL database

In [ ]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [ ]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [ ]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [ ]:
df.to_sql(name="buzz_genius", con=engine, if_exists='replace')